In [ ]:
!pip install pandas numpy matplotlib scipy jinja2 seaborn fastparquet pyarrow statsmodels torch torchvision torchaudio --quiet

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Preprocessing and Feature Extraction

In [ ]:
from gaze_analysis.io_utils import load_data
from gaze_analysis.preprocess import compute_velocity
from gaze_analysis.velocity_analysis import adaptive_saccade_threshold, plot_velocity_distribution
from gaze_analysis.attention_metrics import compute_attention_profile
from gaze_analysis.visualization import (
    plot_attention_timeline, plot_fixations, plot_rolling_entropy
)

import pandas as pd
import numpy as np


path = "data/S_3458_S2_3_VID.csv"
df = load_data(path)
df = compute_velocity(df)
print(f"Sampling rate ≈ {1000/np.mean(np.diff(df['n'])):.1f} Hz")


saccade_thresh = adaptive_saccade_threshold(df)
plot_velocity_distribution(df, saccade_thresh)

profile = compute_attention_profile(df, saccade_thresh)
summary = pd.DataFrame(profile['summary'], index=[0])
display(summary.style.set_caption("Summary of Attention Metrics").background_gradient(axis=None))


plot_attention_timeline(df, saccade_thresh)
plot_fixations(df, profile["fixation_segments"])


times, ents = profile["rolling_entropy_times"], profile["rolling_entropy_values"]
plot_rolling_entropy(times, ents)




In [ ]:
from gaze_analysis.batch_analysis import analyze_directory
summary_df = analyze_directory("data", save_processed=True)

Personalization Analysis

In [ ]:
import importlib
import gaze_analysis.visualization as vis
import gaze_analysis.io_utils as io

importlib.reload(vis)
importlib.reload(io)

In [ ]:
all_df = io.load_processed_files("processed/*_processed.parquet", k=50)
vis.plot_user_vs_group(all_df, user_id="100")

In [ ]:
vis.plot_user_vs_group(all_df, user_id="1002")

In [ ]:
binned = vis._bin_attention(all_df, n_bins=100)
binned.head(), binned.shape, binned.columns

In [ ]:
import gaze_analysis.stat_analysis as stat

intra_df = stat.compute_intra_subject_correlation(binned)
print(intra_df)
print("\nSummary:")
print(intra_df["intra_corr"].describe())


No strong within-user consistency. Eye-movement entropy profiles are not stable across the two video clips.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm

summary = (
    binned.groupby(["participant_id", "session_id"])["rolling_entropy"]
          .mean().reset_index()
)

model = mixedlm("rolling_entropy ~ session_id", summary, groups=summary["participant_id"])
result = model.fit()

print(result.summary())

A mixed-effects analysis was conducted to examine whether participants in the GazeBaseVR dataset exhibited personalized patterns of visual attention, as reflected by their gaze entropy across two video-viewing sessions. The model revealed no significant fixed effect of session (p ≈ 0.997), indicating that average gaze dispersion did not differ between the two video clips. The participant-level random effect (Group Var ≈ 0.005) was small relative to the residual variance (≈ 0.024), suggesting that individual differences accounted for only a minor portion of the total variability in entropy. Together with low intra-subject correlations, these results imply that while some variability exists across participants, gaze entropy patterns are largely consistent across the population and primarily driven by stimulus content rather than stable, person-specific attention traits.

In [ ]:
reg_df = stat.regression_user_vs_group(binned)
print(reg_df.head())
print("\nSummary:")
print(reg_df.describe())

A per-session regression analysis comparing each participant’s gaze entropy trajectory to the group mean revealed moderate individuality in attention dynamics. On average, only about 21 % of a participant’s temporal entropy variation could be explained by the group pattern, indicating that users often deviated from the shared viewing trend. The wide range of slopes (−0.4 – 2.1) and intercepts (−3.3 – 4.1) further suggests that some participants consistently maintained lower entropy—reflecting tighter focus while others exhibited more dispersed gaze behavior. These findings imply that although the stimulus strongly shapes overall attention, individual viewers demonstrate distinct, session-specific gaze signatures.

In [ ]:
vis.plot_user_uniqueness(reg_df)

In [ ]:
from gaze_analysis.data_utils import SlidingGazeDataset

dataset = SlidingGazeDataset(
    "processed/", 
    window_sec=5, 
    pred_sec=5, 
    sample_rate=25, 
    downsample=10, 
    stride=25 
)

print(f"Samples: {len(dataset)}")
x, y = dataset[0]
print("Input shape:", x.shape, "Target shape:", y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Subset

train_idx, test_idx = train_test_split(
    np.arange(len(dataset)), test_size=0.2, random_state=42
)

train_loader = DataLoader(Subset(dataset, train_idx), batch_size=16, shuffle=True)
test_loader  = DataLoader(Subset(dataset, test_idx), batch_size=16, shuffle=False)

print(f"Train: {len(train_idx)} | Test: {len(test_idx)}")

In [ ]:
from gaze_analysis.model import LSTMAttentionForecast
from gaze_analysis.train_utils import train_model

model = LSTMAttentionForecast(input_dim=5, hidden_dim=128, pred_len=125)
trained_model, hist = train_model(model, train_loader, test_loader)

In [ ]:
import matplotlib.pyplot as plt
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
x_batch, y_batch = next(iter(test_loader))
with torch.no_grad():
    y_pred, attn = model(x_batch.to(device))

y_true = y_batch[0].cpu().numpy()
y_hat = y_pred[0].cpu().numpy()

plt.figure(figsize=(10,4))
plt.plot(y_true, label="True (Entropy)")
plt.plot(y_hat, '--', label="Predicted")
plt.title("Next 5s Attention Forecast")
plt.xlabel("Time (samples)")
plt.ylabel("Rolling Entropy")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
from gaze_analysis.evaluate import evaluate_model

r2, corr = evaluate_model(trained_model, test_loader)

I trained an LSTM model with an attention mechanism to predict short-term attention dynamics from gaze data in the GazeBaseVR dataset. Using 5-second input windows of gaze features (x, y, vx, vy, rolling_entropy), the model learned to forecast the next 5 seconds of attention behavior, measured by rolling gaze entropy. After augmenting the dataset with overlapping temporal windows, the model achieved an R² of 0.53 and a Pearson correlation of 0.74 on the test set, indicating that it successfully captured more than half of the variability in attention patterns and accurately followed the temporal trends of gaze entropy. These results suggest that short-term gaze behavior contains meaningful temporal structure and moderate individual predictability, even though long-term consistency across sessions remains limited.